In [1]:
import os
import tensorflow as tf
import json
import numpy as np
import scipy.io
import mat73
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from sklearn.metrics import accuracy_score, precision_score, recall_score


physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
data_dict=mat73.loadmat("D:/Supriyo/onedrive/OneDrive - iitkgp.ac.in/Sanjay_unet/vi.mat")
loc=np.load("D:/Supriyo/onedrive/OneDrive - iitkgp.ac.in/Sanjay_unet/locations.npy")
print("Data loading completed")
data=data_dict['vi']
x_temp=[]
print("segregation inti......")
for i in range(len(loc)):
    
    temp=data[:,loc[i][0]:loc[i][1]+1]
    sh=np.shape(temp)
    i,j=0,0
    while i<=sh[0] and j<=sh[1]:
        x_temp.append(temp[i:i+128,j:j+128])
        i+=128
        j+=128

d:\Supriyo\onedrive\OneDrive - iitkgp.ac.in\Thesis Concrete Work\UNET-FFT\.venv\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Data loading completed
segregation inti......


In [10]:
#Loss

#EDGE Loss TV+sobel
def sobel_edges(image):
    # Define Sobel filters
    sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
    sobel_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32)

    # Calculate Sobel gradients in x and y directions
    grad_x = tf.nn.conv2d(image, sobel_x[..., tf.newaxis, tf.newaxis], strides=[1, 1, 1, 1], padding='SAME')
    grad_y = tf.nn.conv2d(image, sobel_y[..., tf.newaxis, tf.newaxis], strides=[1, 1, 1, 1], padding='SAME')

    # Compute magnitude of gradients
    magnitude = tf.sqrt(grad_x ** 2 + grad_y ** 2)

    return magnitude

def weighted_TV_p_loss(y_true, y_pred, p=0.9):
    loss = tf.reduce_sum(tf.abs(y_true - y_pred) ** p)
    return loss

def edge_loss(y_true, y_pred, sobel_weight, tv_weight, tv_p):
    # Calculate Sobel edges for y_true and y_pred
    sobel_true = sobel_edges(y_true)
    sobel_pred = sobel_edges(y_pred)

    # Calculate the element-wise difference between Sobel edges
    sobel_difference = tf.abs(sobel_true - sobel_pred)

    # Calculate weighted Sobel edge loss
    weighted_sobel_loss = sobel_weight * tf.reduce_mean(sobel_difference)

    # Calculate weighted TV_p loss
    tv_loss = weighted_TV_p_loss(y_true, y_pred, tv_p)

    # Combine the two losses
    edge_loss = weighted_sobel_loss + tv_weight * tv_loss

    return edge_loss

# comined Loss MSE + SSIM
def combined_loss_mse_ssim(y_true, y_pred, ssim_weight=0.5):
    # Calculate SSIM loss
    ssim_loss = 1 - tfa.image.ssim(y_true, y_pred, max_val=1.0)

    # Calculate MSE loss
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

    # Combine both losses with weights
    total_loss = ssim_weight * ssim_loss + (1 - ssim_weight) * mse_loss

    return total_loss

#Fourier Loss
def fourierLoss2(y_actual,y_pred):
    actual_fft = tf.signal.rfft2d(y_actual)
    pred_fft = tf.signal.rfft2d(y_pred)
    lossV=tf.math.real(tf.math.reduce_mean(tf.math.square(actual_fft-pred_fft)))
    return lossV

losses=['mse','mae',tf.keras.losses.CategoricalCrossentropy(),fourierLoss2,weighted_TV_p_loss,edge_loss,combined_loss_mse_ssim]

In [9]:
import tensorflow as tf



def unet(loss,initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9):
    inputs = tf.keras.layers.Input((128, 128, 1))

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    f1 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p1)
    f1 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f1))
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    f2 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p2)
    f2 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f2))
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    f3 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p3)
    f3 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f3))
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    f4 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p4)
    f4 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f4))
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(abs(c6))
    u7 = tf.keras.layers.concatenate([u7, c3])
    u7 = tf.keras.layers.LayerNormalization()(u7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(abs(c7))
    u8 = tf.keras.layers.concatenate([u8, c2])
    u8 = tf.keras.layers.LayerNormalization()(u8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(abs(c8))
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    u9 = tf.keras.layers.LayerNormalization()(u9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='softmax')(c9)
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

    # Learning Rate Decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )
  
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    return model
model = unet(loss='mse')  # Specify the loss function here
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 128, 128, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [11]:
import tensorflow as tf



def unet(loss,initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9):
    inputs = tf.keras.layers.Input((128, 128, 1))

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    f1 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p1)
    f1 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f1))
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    f2 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p2)
    f2 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f2))
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    f3 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p3)
    f3 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f3))
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    f4 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p4)
    f4 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f4))
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(abs(c6))
    u7 = tf.keras.layers.concatenate([u7, c3])
    u7 = tf.keras.layers.LayerNormalization()(u7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(abs(c7))
    u8 = tf.keras.layers.concatenate([u8, c2])
    u8 = tf.keras.layers.LayerNormalization()(u8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(abs(c8))
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    u9 = tf.keras.layers.LayerNormalization()(u9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='softmax')(c9)
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

    # Learning Rate Decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )
  
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    return model
model = unet(loss=losses)  # Specify the loss function here
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 128, 128, 16  160         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_9 (Dropout)            (None, 128, 128, 16  0           ['conv2d_19[0][0]']              
                                )                                                           

In [15]:
import tensorflow_addons as tfa

#Loss

#EDGE Loss TV+sobel
def sobel_edges(image):
    # Define Sobel filters
    sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
    sobel_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32)

    # Calculate Sobel gradients in x and y directions
    grad_x = tf.nn.conv2d(image, sobel_x[..., tf.newaxis, tf.newaxis], strides=[1, 1, 1, 1], padding='SAME')
    grad_y = tf.nn.conv2d(image, sobel_y[..., tf.newaxis, tf.newaxis], strides=[1, 1, 1, 1], padding='SAME')

    # Compute magnitude of gradients
    magnitude = tf.sqrt(grad_x ** 2 + grad_y ** 2)

    return magnitude

def weighted_TV_p_loss(y_true, y_pred, p=0.9):
    loss = tf.reduce_sum(tf.abs(y_true - y_pred) ** p)
    return loss

def edge_loss(y_true, y_pred, sobel_weight, tv_weight, tv_p):
    # Calculate Sobel edges for y_true and y_pred
    sobel_true = sobel_edges(y_true)
    sobel_pred = sobel_edges(y_pred)

    # Calculate the element-wise difference between Sobel edges
    sobel_difference = tf.abs(sobel_true - sobel_pred)

    # Calculate weighted Sobel edge loss
    weighted_sobel_loss = sobel_weight * tf.reduce_mean(sobel_difference)

    # Calculate weighted TV_p loss
    tv_loss = weighted_TV_p_loss(y_true, y_pred, tv_p)

    # Combine the two losses
    edge_loss = weighted_sobel_loss + tv_weight * tv_loss

    return edge_loss

# comined Loss MSE + SSIM
def combined_loss_mse_ssim(y_true, y_pred, ssim_weight=0.5):
    # Calculate SSIM loss
    ssim_loss = 1 - tfa.image.ssim(y_true, y_pred, max_val=1.0)

    # Calculate MSE loss
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

    # Combine both losses with weights
    total_loss = ssim_weight * ssim_loss + (1 - ssim_weight) * mse_loss

    return total_loss

#Fourier Loss
def fourierLoss2(y_actual,y_pred):
    actual_fft = tf.signal.rfft2d(y_actual)
    pred_fft = tf.signal.rfft2d(y_pred)
    lossV=tf.math.real(tf.math.reduce_mean(tf.math.square(actual_fft-pred_fft)))
    return lossV

losses=['mse','mae',tf.keras.losses.CategoricalCrossentropy(),fourierLoss2,weighted_TV_p_loss,edge_loss,combined_loss_mse_ssim]

In [ ]:
val_loss_history = []
val_accuracy_history = []
val_precision_history = []
val_recall_history = []
model=unet()
def on_epoch_end(epoch, logs={}):
    val_loss = logs.get('val_loss')
    val_loss_history.append(val_loss)

    if len(val_loss_history) > patience:
        min_loss = min(val_loss_history[-patience:])
        max_loss = max(val_loss_history[-patience:])
        if max_loss - min_loss < early_stopping_loss:
            print(f"\nEarly stopping at epoch {epoch + 1} as val_loss change < {early_stopping_loss}")
            model.stop_training = True

    # Evaluate the model on validation data after each epoch
    X_val_pred = model.predict(X_val)
    X_val_pred = np.round(X_val_pred)  # Convert probabilities to binary predictions

    # Calculate accuracy, precision, and recall
    accuracy = accuracy_score(X_val.reshape(-1), X_val_pred.reshape(-1))
    precision = precision_score(X_val.reshape(-1), X_val_pred.reshape(-1), average='micro')
    recall = recall_score(X_val.reshape(-1), X_val_pred.reshape(-1), average='micro')

    # Append to history lists
    val_accuracy_history.append(accuracy)
    val_precision_history.append(precision)
    val_recall_history.append(recall)

    # Print the metrics after each epoch
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print(f"{steps_per_epoch}/{steps_per_epoch} [==============================] - {logs.get('loss'):.4f} - accuracy: {accuracy:.4f} - precision: {precision:.4f} - recall: {recall:.4f} - val_loss: {val_loss:.4f}")

# ... (Rest of the code)

# Train the model with early stopping
history = model.fit(
    X_train,
    X_train,  # Use different variables for input and output
    validation_data=(X_val, X_val),  # Add validation data
    callbacks=[model_checkpoint_callback, tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)],
    epochs=100,
    batch_size=batch_size,
    steps_per_epoch=steps_per_epoch
)

In [16]:
x_t = []
for i in x_temp:
    if np.shape(i) == (128, 128):
        x_t.append(i)

scaler = MinMaxScaler()
data = []
for i in x_t:
    data.append(scaler.fit_transform(i))

# Convert data to numpy array
data = np.array(data)

# Split your training data into training and validation sets
X_train, X_val, _, _ = train_test_split(data, data, test_size=0.1, random_state=42)

model = [0, 0, 0, 0, 0, 0]
early_stopping_loss = 4.2e-4  # Loss threshold for early stopping
patience = 5  # Number of epochs with no significant loss improvement before stopping

# Assuming 'losses' is a list of different loss functions
for i in range(len(losses)):
    model[i] = unet(losses[i])
    print(model[i].summary())

    epoch = 0
    checkpoint_path = f"./cp-{losses[i]}-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Calculate the steps per epoch based on your dataset size and batch size
    batch_size = 32  # Set your desired batch size here
    steps_per_epoch = len(X_train) // batch_size

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        mode='min',
        save_freq=5 * steps_per_epoch,  # Save after every 5 epochs
    )

    early_stopping_callback = tf.keras.callbacks.Callback()  # Custom Callback for Early Stopping

    val_loss_history = []

    def on_epoch_end(epoch, logs={}):
        val_loss = logs.get('val_loss')
        val_loss_history.append(val_loss)
        
        if len(val_loss_history) > patience:
            min_loss = min(val_loss_history[-patience:])
            max_loss = max(val_loss_history[-patience:])
            if max_loss - min_loss < early_stopping_loss:
                print(f"\nEarly stopping at epoch {epoch + 1} as val_loss change < {early_stopping_loss}")
                model[i].stop_training = True

    early_stopping_callback.on_epoch_end = on_epoch_end

    # Assuming X_train and X_val are your training and validation datasets, respectively
    history = model[i].fit(
        tf.expand_dims(X_train[:], axis=-1),
        tf.expand_dims(X_train[:], axis=-1),  # Use different variables for input and output
        validation_data=(tf.expand_dims(X_val[:], axis=-1), tf.expand_dims(X_val[:], axis=-1)),  # Add validation data
        callbacks=[model_checkpoint_callback, early_stopping_callback],  # Add early stopping callback
        epochs=100
    )

    with open(f"{losses[i]}.json", "w") as outfile:
        json.dump(history.history, outfile)

    model[i].save(f"model_{losses[i]}.h5")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 128, 128, 16  160         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_9 (Dropout)            (None, 128, 128, 16  0           ['conv2d_19[0][0]']              
                                )                                                           

KeyboardInterrupt: 